In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
import pandas as pd

In [ ]:
def get_row_values(row_data):
    value_list = []
    for record in row_data:
        if '\n\t\t\tN/A\n\t\t' in record.get_text():
            value = None
        else:
            value = record.get_text()
        
        value_list.append(value)
        
    return value_list

def fetch_treasury_yields(year):
    if year == 'ALL':
        treasury_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldAll"
    else:
        treasury_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=" + "%s"% year
        
    page = requests.get(treasury_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    rates_table = soup.find(class_="t-chart")
    
    oddrow_data = get_row_values(rates_table.select(".oddrow .text_view_data"))
    evenrow_data = get_row_values(rates_table.select(".evenrow .text_view_data"))
    
    return {'oddrow': oddrow_data, 'evenrow': evenrow_data}
    

In [ ]:
test_yields = fetch_treasury_yields(2017)

In [ ]:
def is_desc(rates_list):
    rates_list = [i for i in rates_list if i is not None]
    return (sorted(rates_list, reverse=True) == rates_list)

def avg_pct_change(rec_list):
    rec_list = [i for i in rec_list if i is not None]
    pct_change_list = []

    if len(rec_list) == 0:
        return 0

    elif len(rec_list) > 0:
        for x in range(len(rec_list) - 1):
            if rec_list[x] == 0:
                pct_change_list.append(rec_list[x + 1])
            else:
                pct_change_val = (rec_list[x + 1] - rec_list[x]) / rec_list[x]
                pct_change_list.append(pct_change_val * 100)

        return round(mean(pct_change_list), 2)

def cvt_to_float(rec_list):
    float_list = []
    
    for x in rec_list:
        if x is None:
            pass
        else:
            x = float(x)
        
        float_list.append(x)
    
    return float_list
            

def process_row_info(in_list):
    valMap = []
    record_list = []

    counter = 0

    for x in range(len(in_list)):

        record_list.append(in_list[x])
        counter+=1

        if (counter % 13) == 0:
            record_list[0] = datetime.datetime.strptime(record_list[0], "%m/%d/%y").strftime("%Y-%m-%d")
            record_list[1:13] = cvt_to_float(record_list[1:13])

            record_list.extend((is_desc(record_list[1:13]), avg_pct_change(record_list[1:13])))
            valMap.append(record_list)
            record_list = []
            
    return valMap

def combined_scrapped_yields(yield_list):
    if len(yield_list) == 2:

        combined_map = []

        for key in yield_list.keys():
            combined_map.extend(process_row_info(yield_list.get(key)))
            
        return combined_map

In [ ]:
from statistics import mean
y = combined_scrapped_yields(test_yields)

In [ ]:
processed_yields = sorted(y, key=lambda x: x[0])

In [ ]:
processed_yields[0:2]

In [ ]:
import mariadb

In [ ]:
import sys

# Instantiate Connection
try:
   conn = mariadb.connect(
      user="root",
      password="root",
      host="localhost",
       db='us_treasury_yield_rates',
      port=3306)
except mariadb.Error as e:
   print(f"Error connecting to MariaDB Platform: {e}")
   sys.exit(1)

In [ ]:
# Instantiate Cursor
cur = conn.cursor(buffered=True , dictionary=True)

In [ ]:
var_string = ', '.join('?' * len(processed_yields[0]))
var_string

In [ ]:
for records in processed_yields:
    query_string = 'INSERT INTO us_treasury_yield_rates.test_rates VALUES (%s);' % var_string
    cur.execute(query_string, records)
    conn.commit()

In [ ]:
query = f"SELECT * FROM us_treasury_yield_rates.test_rates"

cur.execute(query)

rows = cur.fetchall()

conn.close()


In [ ]:
result = []

for x in range(len(rows)):
    #print('now it is: ' + str(x))
    single_query = []

    for key in rows[x].keys():
        rec = str(rows[x].get(key))
        single_query.append(rec)

    result.append(single_query)
            
print(result[0:2])

In [ ]:
import wbgapi as wb

In [ ]:
indicators = {'NY.GDP.MKTP.CD':'GDP (current US$)'}

df = wb.data.DataFrame(indicators, 'USA', time=range(2000, 2020))

In [ ]:
year_range = ""
for x in range(1990,2021):
    year_range += (str(x) + ',')
    
year_range

In [ ]:
bea_url = "https://apps.bea.gov/api/data/?&UserID=<API KEY>&method=GetData&DataSetName=NIPA&TableName=T10105&SeriesCode=A191RC&LineDescription=Gross+domestic+product&Frequency=Q&Year=" + year_range + "&ResultFormat=json"
# data in millions

page2 = requests.get(bea_url)


In [ ]:
import json
from urllib.request import urlopen
response = urlopen(bea_url)
data_json = json.loads(response.read())

In [ ]:
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

raw_q_gdp = []

for x in data_json['BEAAPI']['Results']['Data']:
    
    if 'Gross domestic product' in x.values():
        y_gdp = []
        y_gdp.append(x['TimePeriod'])
        y_gdp.append(locale.atoi(x['DataValue']))
        #y_gdp.append(locale.atof(x['DataValue']))
        raw_q_gdp.append(y_gdp)

In [ ]:
raw_q_gdp

In [ ]:
def assign_dates(qlist):
    vals = { 'Q1' : '03-30'  ,  'Q2' : '06-30'  ,  'Q3' : '09-30', 'Q4' : '12-30' }
    
    for x in range(len(qlist)):
        cal_date = qlist[x][0][0:4] + "-" + vals[qlist[x][0][-2:]]
        qlist[x].extend([cal_date])
    
    return qlist

In [ ]:
assign_dates(raw_q_gdp)

In [ ]:
def mil_to_tril(qlist):
    
    for x in range(len(qlist)):
        qlist[x].extend([round(qlist[x][1] / (10**6),2)])
    
    return qlist

In [ ]:
mil_to_tril(raw_q_gdp)[0]

In [ ]:
var_string2 = ', '.join('?' * len(raw_q_gdp[0]))
var_string2

In [ ]:
for records in raw_q_gdp:
    query_string = 'INSERT INTO us_treasury_yield_rates.test_gdp VALUES (%s);' % var_string2
    cur.execute(query_string, records)
    conn.commit()

In [ ]:
my_str =  "abcdefghij"
my_str = my_str[:-1]

In [ ]:
year_range[:-1]

In [5]:
import yfinance as yf

sp500 = yf.Ticker("^GSPC")

In [1]:
import yfinance as yf
data = yf.download("^GSPC", start="2021-01-01", end="2021-07-02")

[*********************100%***********************]  1 of 1 completed


In [10]:
col_list = ['Open', 'Close']
new_list = data[col_list]

In [11]:
new_list.reset_index(level=0, inplace=True)

In [12]:
new_list

,Date,Open,Close
0,2020-12-31,3733.270020,3756.070068
1,2021-01-04,3764.610107,3700.649902
2,2021-01-05,3698.020020,3726.860107
3,2021-01-06,3712.199951,3748.139893
4,2021-01-07,3764.709961,3803.790039
...,...,...,...
121,2021-06-25,4274.450195,4280.700195
122,2021-06-28,4284.899902,4290.609863
123,2021-06-29,4293.209961,4291.799805
124,2021-06-30,4290.649902,4297.500000


In [ ]:
new_list['Date'] = [str(item)[0:10] for item in new_list['Date']]
new_list['Open'] = [round(item,2) for item in new_list['Open']]
new_list['Close'] = [round(item,2) for item in new_list['Close']]
new_list['Ticker'] = 'S&P 500'
new_list['Type'] = 'Index'


In [ ]:
new_list

In [ ]:
sp_list = new_list.values.tolist()

In [ ]:
sp_list[0]

In [ ]:
var_string3 = ', '.join('?' * len(sp_list[0]))
var_string3

for records in sp_list:
    query_string = 'INSERT INTO us_treasury_yield_rates.us_stock_market VALUES (%s);' % var_string3
    cur.execute(query_string, records)
    conn.commit()
